First use requests and BeautifulSoup to read the html of the Toronto postal codes into variable soup

In [1]:
import requests
from bs4 import BeautifulSoup
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Use BeautifulSoup to read the html of the postalcode table into variable table. 
Then read the body of the table into variable table_body. 
Next, read the table rows into variable table_rows.
Then iterate through the table rows to read the text of the table elements (td) into the variable records. 
Display the contents of variable records

In [2]:
table=soup.find('table', class_='wikitable sortable')
table_body=table.find('tbody')
table_rows=table_body.find_all('tr')

del table_rows[0]         # delete empty row coming from header

records = []
for row in table_rows:
    cells = row.find_all('td')
    cells=[cell.text.strip() for cell in cells]   # find text inside td
    records.append(cells)

records

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 

Set up a pandas dataframe variable df with column headers 'PostalCode', 'Borough', and 'Neighborhood'. Fill df with the contents of records. Where the value of 'Neighborhood' is 'Not assigned' then change that value to the value of 'Borough'. Group by 'PostalCode' and 'Borough' and combine 'Neighborhood' values separated by commas. Display the dataframe df.

In [3]:
import pandas as pd
headers = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(records, columns=headers)
df = df[df['Borough'] != 'Not assigned']

# where neighborhood == 'Not assigned' make Neighborhood = Borough
cols = df.columns[df.columns.str.contains('Neighborhood')]
df[cols] = df[cols].mask(df[cols].apply(lambda x: x.str.contains('Not assigned')), df['Borough'], axis=0)

df = df.groupby(['PostalCode','Borough'], sort=False).agg(lambda x: ', '.join(x)).reset_index()
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Print the number of rows of the dataframe

In [4]:
df.shape

(103, 3)

Read in latitudes and longitudes postal codes and rename column with postal codes to allow for merge with main dataframe

In [5]:
df_lat_long = pd.read_csv("https://cocl.us/Geospatial_data")
df_lat_long.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df = pd.merge(df, df_lat_long, on='PostalCode', sort=False)
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


Get the geographical coordinates of Toronto and visualize the boroughs with various postal codes

In [6]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

import folium
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Solving environment: done

# All requested packages already installed.



Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'X4W2C144TDSEFHIJ0WOHPPGFMDLB5XX5PGWOBK1W5F1HUNXG' # Foursquare ID
CLIENT_SECRET = 'FWXABKV0LGGCBVUTQJWWAC2N0WDDZL5CHNO2EROP0QPDO1WM' # Foursquare Secret
VERSION = '20180605' # Foursquare API version


Define function to explore nearby venues in boroughs

In [8]:
LIMIT=100
radius=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

run the above function on each borough and create a new dataframe called toronto_venues.

In [9]:
toronto_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

check resulting dataframe

In [10]:
print(toronto_venues.shape)
toronto_venues.head()

(2243, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,North York,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


check how many venues were returned for each borough

In [11]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,114,114,114,114,114,114
Downtown Toronto,1284,1284,1284,1284,1284,1284
East Toronto,123,123,123,123,123,123
East York,74,74,74,74,74,74
Etobicoke,69,69,69,69,69,69
Mississauga,11,11,11,11,11,11
North York,236,236,236,236,236,236
Queen's Park,42,42,42,42,42,42
Scarborough,91,91,91,91,91,91


find out how many unique categories can be curated from all the returned venues

In [12]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 279 uniques categories.


Analyze each borough

In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Examine the new dataframe size

In [14]:
toronto_onehot.shape

(2243, 280)

Group rows by borough and by taking the mean of the frequency of occurrence of each category

In [15]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.008772,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.008772
1,Downtown Toronto,0.000000,0.000779,0.000779,0.000779,0.000779,0.000779,0.001558,0.001558,0.001558,...,0.000000,0.007788,0.002336,0.000000,0.004673,0.000000,0.006231,0.000779,0.001558,0.003894
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013514,0.000000,0.013514,0.000000,0.000000,0.000000,0.013514
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004237,0.000000,0.000000,0.004237,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.004237,0.004237,0.008475,0.000000,0.000000,0.004237,0.004237,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.023810
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011111,0.000000,0.000000,0.011111,0.000000,0.005556,0.000000,0.000000,0.005556


Confirm the new size

In [16]:
toronto_grouped.shape

(11, 280)

Print each borough along with the top 5 most common venues

In [17]:
num_top_venues = 5

for borough in toronto_grouped['Borough']:
    print("----"+borough+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.07
2            Café  0.05
3            Park  0.05
4     Pizza Place  0.05


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Restaurant  0.04
3       Bakery  0.03
4        Hotel  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.08
1         Coffee Shop  0.07
2      Ice Cream Shop  0.04
3  Italian Restaurant  0.04
4                Café  0.03


----East York----
                 venue  freq
0          Coffee Shop  0.07
1                 Bank  0.04
2                 Park  0.04
3  Sporting Goods Shop  0.04
4        Grocery Store  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.12
1  Sandwich Place  0.07
2        Pharmacy  0.06
3     Coffee Shop  0.06
4            Park  0.04


----Mississauga----
                 venue  freq
0          Coffee Shop  0.18
1                Hotel  0.18
2       Sandwich Plac

function to sort venues in descending order

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create new dataframe and display top 10 venues for each borough

In [19]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Burger Joint,Clothing Store,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Pizza Place
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Brewery,Sandwich Place,Bookstore,Pizza Place,American Restaurant
3,East York,Coffee Shop,Bank,Burger Joint,Sporting Goods Shop,Grocery Store,Pizza Place,Park,Pharmacy,Pet Store,Sandwich Place
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Fast Food Restaurant,Park,Gym,Grocery Store,Flower Shop,Convenience Store


Run k-means to cluster boroughs

In [20]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 4, 1, 0, 3, 4, 0], dtype=int32)

create new dataframe that includes the cluster as well as the top 10 venues for each borough

In [21]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



In [22]:
toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Pizza Place
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,3,Coffee Shop,Gym,Sushi Restaurant,Japanese Restaurant,Diner,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Bubble Tea Shop


Visualize the resulting clusters

In [23]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters

In [ ]:
Cluster 1

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
1,North York,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
2,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Pizza Place
3,North York,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
7,North York,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
9,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Pizza Place
10,North York,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
13,North York,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
15,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Pizza Place
19,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Brewery,Sandwich Place,Bookstore,Pizza Place,American Restaurant


Cluster 2

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,Mississauga,1,Coffee Shop,Hotel,American Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Fried Chicken Joint,Mediterranean Restaurant,Burrito Place,Sandwich Place,Dog Run


Cluster 3

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,York,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant
21,York,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant
56,York,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant
63,York,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant
64,York,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant


Cluster 4

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Queen's Park,3,Coffee Shop,Gym,Sushi Restaurant,Japanese Restaurant,Diner,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Bubble Tea Shop


Cluster 5

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Fast Food Restaurant,Park,Gym,Grocery Store,Flower Shop,Convenience Store
6,Scarborough,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
8,East York,4,Coffee Shop,Bank,Burger Joint,Sporting Goods Shop,Grocery Store,Pizza Place,Park,Pharmacy,Pet Store,Sandwich Place
11,Etobicoke,4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Fast Food Restaurant,Park,Gym,Grocery Store,Flower Shop,Convenience Store
12,Scarborough,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
14,East York,4,Coffee Shop,Bank,Burger Joint,Sporting Goods Shop,Grocery Store,Pizza Place,Park,Pharmacy,Pet Store,Sandwich Place
17,Etobicoke,4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Fast Food Restaurant,Park,Gym,Grocery Store,Flower Shop,Convenience Store
18,Scarborough,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
22,Scarborough,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
23,East York,4,Coffee Shop,Bank,Burger Joint,Sporting Goods Shop,Grocery Store,Pizza Place,Park,Pharmacy,Pet Store,Sandwich Place
